# Se importan las bibliotecas necesarias

In [1]:
import pandas as pd
import re
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
import lightgbm as lgb
from catboost import CatBoostRegressor
%matplotlib inline

# Se lee el .csv

In [2]:
train = pd.read_csv('csv/train.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [3]:
train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
res_modelo2 = pd.read_csv('csv/yugioh.csv')
res_modelo2.head()

,Unnamed: 0,0
0,0,0.321528
1,1,0.694210
2,2,0.032789
3,3,0.128172
4,4,0.866353


In [5]:
del res_modelo2['Unnamed: 0']
res_modelo2.head()

,0
0,0.321528
1,0.694210
2,0.032789
3,0.128172
4,0.866353


In [6]:
train['res_modelo2'] = res_modelo2.iloc[:, 0]
train.head()

,id,keyword,location,text,target,res_modelo2
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.694210
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.032789
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.128172
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.866353


# Elaboración del modelo

Para este modelo se utilizarán árboles de decisión para realizar las predicciones y se tendrán en cuenta los siguientes features extraídos del análisis exploratorio, los cuales fueron seleccionados por el grupo como aquellos que podrían resultan más interesantes:

- Palabras con mayor y menor porcentaje de veracidad.
- Pares de 2 palabras con mayor y menor porcentaje de veracidad.
- Tweets que contienen @, ¿?, ¡! tienden a ser falsos.
- Keywords con mayor y menor porcentaje de veracidad.
- Locaciones más y menos veraces.
- Longitud.

### Preparación del set de datos

#### Parte inicial

In [7]:
#Palabras con mayor y menor porcentaje de veracidad
min_repetitions = ((0.2/100)*len(train.index))
words = train['text'].str.split()
clean_words = []

for sentence in words:
    clean_sentence = []
    for word in sentence:
        clean_word = re.sub('[^A-Za-z0-9]+','', word)
        if(clean_word != ''):
            clean_sentence.append(clean_word.lower())
    clean_words.append(clean_sentence)

In [8]:
train_aux = train
train_aux['words'] = clean_words
train_aux = train_aux.explode('words')
train_aux.head()

,id,keyword,location,text,target,res_modelo2,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,our
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,reason


In [9]:
train_aux = train_aux.groupby('words').agg({'target':['sum','count']})
train_aux = train_aux[train_aux['target']['count']>min_repetitions]
train_aux['veracity'] = train_aux['target']['sum']/train_aux['target']['count']
train_aux.head()

target        veracity
         sum count          
words                       
05        14    18  0.777778
1         29    64  0.453125
10         9    23  0.391304
100        6    17  0.352941
11         7    21  0.333333

In [10]:
train_aux.reset_index(inplace=True) 

In [11]:
#Pares de 2 palabras con mayor y menor porcentaje de veracidad.
train_aux2 = train
train_aux2['words'] = clean_words
words_pairs = []

for sentence in train_aux2['words']:
    pairs = []
    for i in range(len(sentence)-1):
        pairs.append(sentence[i] + ' ' + sentence[i+1])
    words_pairs.append(pairs)
    
train_aux2['words'] = words_pairs
train_aux2 = train_aux2.explode('words')
train_aux2.head()

,id,keyword,location,text,target,res_modelo2,words
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,our deeds
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,deeds are
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,are the
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,the reason
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,reason of


In [12]:
train_aux2 = train_aux2.groupby('words').agg({'target':['sum','count']})
train_aux2 = train_aux2[train_aux2['target']['count']>min_repetitions]
train_aux2['veracity'] = train_aux2['target']['sum']/train_aux2['target']['count']

In [13]:
#las palabras que son '' la cagan
train_aux2.head(10)

target        veracity
                 sum count          
words                               
11yearold boy     20    20  1.000000
16yr old          28    28  1.000000
40 families       26    26  1.000000
70 years          30    31  0.967742
a blast            3    16  0.187500
a car              9    16  0.562500
a cliff            8    24  0.333333
a few              8    18  0.444444
a fire             8    16  0.500000
a good             5    16  0.312500

In [14]:
train_aux2.reset_index(inplace=True)

In [15]:
# Keywords más y menos veraces
train_aux3 = train
train_aux3 = train_aux3.groupby('keyword').agg({'target':['sum','count']})
train_aux3 = train_aux3[train_aux3['target']['count']>min_repetitions]
train_aux3['veracity'] = train_aux3['target']['sum']/train_aux3['target']['count']
train_aux3.head()

target        veracity
                       sum count          
keyword                                   
ablaze                  13    36  0.361111
accident                24    35  0.685714
aftershock               0    34  0.000000
airplane%20accident     30    35  0.857143
ambulance               20    38  0.526316

In [16]:
train_aux3.reset_index(inplace=True)

In [17]:
# Locaciones más y menos veraces
train_aux4 = train
train_aux4 = train_aux4.groupby('location').agg({'target':['sum','count']})
train_aux4 = train_aux4[train_aux4['target']['count']>min_repetitions]
train_aux4['veracity'] = train_aux4['target']['sum']/train_aux4['target']['count']
train_aux4.head()

target        veracity
               sum count          
location                          
Australia        9    18  0.500000
California       7    17  0.411765
Canada          13    29  0.448276
Chicago, IL      9    18  0.500000
India           20    24  0.833333

In [18]:
train_aux4.reset_index(inplace=True)

#### Parte final

In [19]:
top_words = train_aux[train_aux['veracity']>0.9]['words']
worst_words = train_aux[train_aux['veracity']<0.1]['words']

In [20]:
top_words_pairs = train_aux2[train_aux2['veracity']>0.9]['words']
worst_words_pairs = train_aux2[train_aux2['veracity']<0.1]['words']

In [21]:
top_keywords = train_aux3[train_aux3['veracity']>0.9]['keyword']
worst_keywords = train_aux3[train_aux3['veracity']<0.1]['keyword']

In [22]:
top_locations = train_aux4[train_aux4['veracity']>0.9]['location']
worst_locations = train_aux4[train_aux4['veracity']<0.1]['location']

In [23]:
train = train.drop(columns='words')
train['long(char)'] = train['text'].str.len()

aux = []
for i in words:
    aux.append(len(i))

train['long(word)'] = aux
train.head()

,id,keyword,location,text,target,res_modelo2,long(char),long(word)
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,69,13
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.694210,38,7
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.032789,133,22
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.128172,65,8
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.866353,88,16


In [24]:
def count_text_ocurrences(texts, words):
    ocurrences = []
    for text in texts:
        count = 0
        for word in words:
            if word in text:
                count = count+1
        ocurrences.append(count)
    return ocurrences

def count_ocurrences(searched, series):
    ocurrences = []
    for element in searched:
        count = 0
        if element in series:
            count = count+1
        ocurrences.append(count)
    return ocurrences

def contains_char(texts, char1, char2=''):
    ocurrences = []
    for text in texts:
        count = 0
        if char1 in text:
            count = 1
        if (count==0)&(char2!=''):
            if char2 in text:
                count = 1
        ocurrences.append(count)
    return ocurrences

In [25]:
train['top_words'] = count_text_ocurrences(train['text'], top_words)
train['worst_words'] = count_text_ocurrences(train['text'], worst_words)
train['top_words_pairs'] = count_text_ocurrences(train['text'], top_words_pairs)
train['worst_words_pairs'] = count_text_ocurrences(train['text'], worst_words_pairs)
train.head()

,id,keyword,location,text,target,res_modelo2,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,69,13,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.694210,38,7,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.032789,133,22,1,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.128172,65,8,1,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.866353,88,16,1,0,0,0


In [26]:
train['top_keywords'] = count_ocurrences(train['keyword'], top_keywords)
train['worst_keywords'] = count_ocurrences(train['keyword'], worst_keywords)
train['top_locations'] = count_ocurrences(train['location'], top_locations)
train['worst_locations'] = count_ocurrences(train['location'], worst_locations)
train.head()

,id,keyword,location,text,target,res_modelo2,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,69,13,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.694210,38,7,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.032789,133,22,1,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.128172,65,8,1,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.866353,88,16,1,0,0,0,0,0,0,0


In [27]:
train['arroba'] = contains_char(train['text'],'@')
train['singnosDeInterrogacion'] = contains_char(train['text'],'\?', '¿')
train['signosDeExclamacion'] = contains_char(train['text'],'!','¡')
train.head()

,id,keyword,location,text,target,res_modelo2,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,arroba,singnosDeInterrogacion,signosDeExclamacion
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,0.321528,69,13,0,0,0,0,0,0,0,0,0,0,0
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,0.694210,38,7,0,0,0,0,0,0,0,0,0,0,0
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,0.032789,133,22,1,0,0,0,0,0,0,0,0,0,0
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,0.128172,65,8,1,0,0,0,0,0,0,0,0,0,0
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,0.866353,88,16,1,0,0,0,0,0,0,0,0,0,0


In [28]:
train = train.drop(columns=['keyword','location','text'])
train = train.set_index('id')
train.head(20)

,target,res_modelo2,long(char),long(word),top_words,worst_words,top_words_pairs,worst_words_pairs,top_keywords,worst_keywords,top_locations,worst_locations,arroba,singnosDeInterrogacion,signosDeExclamacion
id,,,,,,,,,,,,,,,
1,1,0.321528,69,13,0,0,0,0,0,0,0,0,0,0,0
4,1,0.694210,38,7,0,0,0,0,0,0,0,0,0,0,0
5,1,0.032789,133,22,1,0,0,0,0,0,0,0,0,0,0
6,1,0.128172,65,8,1,0,0,0,0,0,0,0,0,0,0
7,1,0.866353,88,16,1,0,0,0,0,0,0,0,0,0,0
8,1,0.364047,110,18,1,0,0,0,0,0,0,0,0,0,0
10,1,0.978321,95,14,0,0,0,0,0,0,0,0,0,0,0
13,1,0.912725,59,15,0,0,1,0,0,0,0,0,0,0,0
14,1,0.673439,79,12,0,0,0,0,0,0,0,0,0,0,0


In [29]:
col_rename = {'long(char)': 'longEnChars', 'long(word)': 'longEnWords', 'top_words': 'topWords', 'worst_words': 'worstWords', 'top_words_pairs': 'topWordsPairs',  
             'worst_words_pairs': 'worstWordsPairs', 'top_keywords': 'topKeywords', 'worst_keywords': 'worstKeywords', 'top_locations': 'topLocations', 
             'worst_locations': 'worstLocations'}
train = train.rename(columns= col_rename)
train.head(20)

,target,res_modelo2,longEnChars,longEnWords,topWords,worstWords,topWordsPairs,worstWordsPairs,topKeywords,worstKeywords,topLocations,worstLocations,arroba,singnosDeInterrogacion,signosDeExclamacion
id,,,,,,,,,,,,,,,
1,1,0.321528,69,13,0,0,0,0,0,0,0,0,0,0,0
4,1,0.694210,38,7,0,0,0,0,0,0,0,0,0,0,0
5,1,0.032789,133,22,1,0,0,0,0,0,0,0,0,0,0
6,1,0.128172,65,8,1,0,0,0,0,0,0,0,0,0,0
7,1,0.866353,88,16,1,0,0,0,0,0,0,0,0,0,0
8,1,0.364047,110,18,1,0,0,0,0,0,0,0,0,0,0
10,1,0.978321,95,14,0,0,0,0,0,0,0,0,0,0,0
13,1,0.912725,59,15,0,0,1,0,0,0,0,0,0,0,0
14,1,0.673439,79,12,0,0,0,0,0,0,0,0,0,0,0


### Entrenamiento del set de datos

In [39]:
for n in range (2, len(train.columns), 1):
    for m in range (n+1, len(train.columns), 1):
        for g in range (m+1, len(train.columns), 1):
            X, y = train.iloc[:,[1,n,m,g]], train.iloc[:,0]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=465)
            xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                            colsample_bytree = 0.3, learning_rate = 0.1,
                            max_depth = 20, alpha = 10, n_estimators = 80)
            xg_reg.fit(X_train,y_train)
            preds = xg_reg.predict(X_test)
            if(accuracy_score(y_test,preds.round()) > 0.65):
                print(str(n) + ' y ' + str(m) + ' y '+ str(g))
                print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
                print('Precision Score : ' + str(precision_score(y_test,preds.round())))
                print('Recall Score : ' + str(recall_score(y_test,preds.round())))
                print('F1 Score : ' + str(f1_score(y_test,preds.round())))

2 y 3 y 4
Accuracy Score : 0.660245183887916
Precision Score : 0.708029197080292
Recall Score : 0.38645418326693226
F1 Score : 0.5
2 y 4 y 5
Accuracy Score : 0.6611208406304728
Precision Score : 0.697594501718213
Recall Score : 0.4043824701195219
F1 Score : 0.5119798234552333
2 y 4 y 7
Accuracy Score : 0.6541155866900175
Precision Score : 0.6777408637873754
Recall Score : 0.4063745019920319
F1 Score : 0.5080946450809465
2 y 4 y 12
Accuracy Score : 0.6619964973730298
Precision Score : 0.7265625
Recall Score : 0.3705179282868526
F1 Score : 0.49076517150395776
2 y 4 y 14
Accuracy Score : 0.6532399299474606
Precision Score : 0.6853146853146853
Recall Score : 0.3904382470119522
F1 Score : 0.4974619289340101
3 y 4 y 5
Accuracy Score : 0.6514886164623468
Precision Score : 0.7429906542056075
Recall Score : 0.31673306772908366
F1 Score : 0.4441340782122905
3 y 4 y 12
Accuracy Score : 0.6506129597197898
Precision Score : 0.764102564102564
Recall Score : 0.2968127490039841
F1 Score : 0.4275466284

#### XGBoost

In [ ]:
xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 20, alpha = 10, n_estimators = 80)

In [ ]:
xg_reg.fit(X_train,y_train)

In [ ]:
preds = xg_reg.predict(X_test)

In [ ]:
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
plt.rcParams['figure.figsize'] = [12, 7]
xgb.plot_importance(xg_reg)

In [ ]:
# params = {
#         'min_child_weight': [1, 5, 10],
#         'gamma': [0.5, 1, 1.5, 2, 5],
#         'subsample': [0.6, 0.8, 1.0],
#         'colsample_bytree': [0.2, 0.3, 0.4],
#         'max_depth': [10, 15, 20]
#         }
# grid_acc = GridSearchCV(xg_reg, param_grid = params)
# grid_acc.fit(X_train, y_train)
# y_pred_acc = grid_acc.predict(X_test)

# # New Model Evaluation metrics 
# print('Accuracy Score : ' + str(accuracy_score(y_test,y_pred_acc.round())))
# print('Precision Score : ' + str(precision_score(y_test,y_pred_acc.round())))
# print('Recall Score : ' + str(recall_score(y_test,y_pred_acc.round())))
# print('F1 Score : ' + str(f1_score(y_test,y_pred_acc.round())))

#### RandomForest

In [ ]:
rf_model = RandomForestRegressor(random_state=13, n_estimators=80, max_depth=20)
rf_model.fit(X_train, y_train)
preds = rf_model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

rf_model.feature_importances_
plt.figure(figsize=(17,9))
plt.bar(X_train.columns, rf_model.feature_importances_)
plt.xlabel('Features')
plt.ylabel('Importancia')
plt.title('Importancia Features con RF')
plt.show()

#### LightGBM

In [ ]:
lgb_train = lgb.Dataset(X_train, y_train)
lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

params = {
     'objective': 'regression',
     'metric': 'rmse',
     'num_leaves': 5,
     'learning_rate': 0.1,
     'feature_fraction': 0.9,
 }

params = {
     'objective': 'regression',
 }

gbm = lgb.train(params,
                 lgb_train,
                 num_boost_round=100,
                 valid_sets=lgb_eval,
                 early_stopping_rounds=10)

preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))
rmse = np.sqrt(mean_squared_error(y_test, preds))
print("RMSE: %f" % (rmse))

#### CatBoost

In [ ]:
model = CatBoostRegressor()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)
preds = model.predict(X_test)

print('Accuracy Score : ' + str(accuracy_score(y_test,preds.round())))
print('Precision Score : ' + str(precision_score(y_test,preds.round())))
print('Recall Score : ' + str(recall_score(y_test,preds.round())))
print('F1 Score : ' + str(f1_score(y_test,preds.round())))